Hello this note book is our first attempt at text generation.
THhs is an intro to nlp and the dangers of ai and ethics all in one

In [1]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
import tensorflow.keras.utils as ku 
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional, Embedding, GRU
from keras.utils import np_utils
from tensorflow.keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Reading and cleanning data so that we take only the unique tweets and remove all links in them. we also make sure that after we do that we remove any that are empty strings. additonally remove all double quotes

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Data/tweets/trump_insult_tweets_2014_to_2021.csv',index_col='Unnamed: 0')
df.describe()

,date,target,insult,tweet
count,10360,10358,10360,10360
unique,1573,866,6729,5673
top,2020-10-12,the-media,Fake News,"The Fake News Networks, those that knowingly h..."
freq,45,1287,431,16


In [4]:
df['clean_tweets'] = df['tweet'].apply(lambda x: re.sub(r'http\S+', '', str(x)))

In [5]:
clean_tweets = np.array(df['clean_tweets'][df['clean_tweets'] != ''].unique())

In [6]:
np.array([len(tweet) for tweet in clean_tweets]).argmin()

5254

In [7]:
clean_tweets = [x.replace('"','') for x in clean_tweets]

Tokenizing the tweets. our model will take an input of (5=seq_len) word hashes and predict the next word we will try to do a different n-grams with padding and see if its better in the next model. 

In [8]:
tokenizer = Tokenizer()
corpus = clean_tweets
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1
print(total_words)

10385


In [9]:
token_list = tokenizer.texts_to_sequences([corpus[0]])[0]
print(token_list)
print([corpus[0]])

[111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304, 36, 706, 380, 19, 5740, 56, 16, 444, 73, 11, 17, 66, 7, 1384, 5741, 1217]
['Can you believe this fool, Dr. Thomas Frieden of CDC, just stated, anyone with fever should be asked if they have been in West Africa DOPE']


In [10]:

seq_len = 5
X_data = []
y_data = []
input_sequences = []

for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)
        


In [11]:
input_sequences[:20]

[[111, 45],
 [111, 45, 243],
 [111, 45, 243, 27],
 [111, 45, 243, 27, 826],
 [111, 45, 243, 27, 826, 1530],
 [111, 45, 243, 27, 826, 1530, 4303],
 [111, 45, 243, 27, 826, 1530, 4303, 5739],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304, 36],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304, 36, 706],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304, 36, 706, 380],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304, 36, 706, 380, 19],
 [111, 45, 243, 27, 826, 1530, 4303, 5739, 6, 4304, 36, 706, 380, 19, 5740],
 [111,
  45,
  243,
  27,
  826,
  1530,
  4303,
  5739,
  6,
  4304,
  36,
  706,
  380,
  19,
  5740,
  56],
 [111,
  45,
  243,
  27,
  826,
  1530,
  4303,
  5739,
  6,
  4304,
  36,
  706,
  380,
  19,
  5740,
  56,
  16],
 [111,
  45,
  243,
  27,
  826,
  1530,
  4303,
  5739,
  6,
  4304,
  36,
  706,
  380,
  19,
  5740,
  56,
  16,
  444],
 [111,
  45,
  243,
  27

In [12]:
# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [13]:
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

In [14]:
predictors.shape, label.shape

((180067, 59), (180067, 10385))

In [15]:
predictors[:10], label[:10]

(array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  111],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,  111,   45],
        [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,

In [16]:
#n_patterns = len(X_data) #157566 seq_length = 5
#pd.DataFrame(y_data).iloc[:,0].sort_values().unique()


In [17]:
# X = np.reshape(X_data, (n_patterns, seq_len, 1))
# X = X/float(len(tokenizer.word_index.items()))
# y = np.asarray(pd.get_dummies(y_data))


In [18]:
#X.shape, y.shape, len(tokenizer.word_index.items())

In [19]:
# print(y[0][1529])
# len(X_data[0])

In [31]:
model = Sequential()

model.add(Embedding(total_words, 200, input_length=max_sequence_len-1))

model.add(Bidirectional(GRU(512, return_sequences=True)))
model.add(Dropout(0.2))
model.add(GRU(512))
model.add(Dropout(0.2))

model.add(Dense(total_words, activation='softmax')) #outputs a on hot encoded row 
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 59, 200)           2077000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 59, 1024)          2193408   
_________________________________________________________________
dropout_6 (Dropout)          (None, 59, 1024)          0         
_________________________________________________________________
gru_7 (GRU)                  (None, 512)               2362368   
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10385)             5327505   
Total params: 11,960,281
Trainable params: 11,960,281
Non-trainable params: 0
__________________________________________

In [32]:
#filename = "GRU_model_weights_saved.hdf5"
#model.load_weights(filename)
#model.compile(loss='categorical_crossentropy', optimizer='adam')

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
filepath = "/content/drive/MyDrive/Colab Data/tweets/GRU512model_weights_saved_long.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
desired_callbacks = [checkpoint]


In [34]:
model.fit(predictors, label, epochs=150, batch_size=2000, callbacks=desired_callbacks)

Epoch 1/150
91/91 [==============================] - 41s 410ms/step - loss: 7.5500 - accuracy: 0.0438

Epoch 00001: loss improved from inf to 7.10469, saving model to /content/drive/MyDrive/Colab Data/tweets/GRU512model_weights_saved_long.hdf5
Epoch 2/150
91/91 [==============================] - 37s 409ms/step - loss: 6.8537 - accuracy: 0.0484

Epoch 00002: loss improved from 7.10469 to 6.87007, saving model to /content/drive/MyDrive/Colab Data/tweets/GRU512model_weights_saved_long.hdf5
Epoch 3/150
91/91 [==============================] - 37s 408ms/step - loss: 6.8587 - accuracy: 0.0477

Epoch 00003: loss improved from 6.87007 to 6.86620, saving model to /content/drive/MyDrive/Colab Data/tweets/GRU512model_weights_saved_long.hdf5
Epoch 4/150
91/91 [==============================] - 37s 409ms/step - loss: 6.8597 - accuracy: 0.0476

Epoch 00004: loss improved from 6.86620 to 6.86596, saving model to /content/drive/MyDrive/Colab Data/tweets/GRU512model_weights_saved_long.hdf5
Epoch 5/150


In [ ]:
model.save('/content/drive/MyDrive/Colab Data/tweets/GRU512trump_model.h5')

#model.save_weights('GRUtrump_weights.hdf5')

In [ ]:
pattern = [6,
  4304,
  36,
  706,
  380] 
pattern = np.array(pattern)

#model.summary()
#x = np.reshape(pattern, (1,len(pattern), 1))
print(pattern.shape)
print(pattern)

(5,)
[   6 4304   36  706  380]


In [ ]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

# Function takes a tokenized sentence and returns the words
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(word) for word in list_of_indices]
    return(words)

# Creating texts 
my_texts = sequence_to_text(pattern)
print(my_texts)
#tokenizer.sequences_to_texts_generator([pattern])

['of', 'cdc', 'just', 'stated', 'anyone']


In [ ]:
seed_text = "We are"
next_words = 50
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
    #print(token_list)
    predicted = model.predict(token_list, verbose=0)

    output_word = ""
    
    for word, index in tokenizer.word_index.items():
        if index == predicted.argmax():
            output_word = word
            break
    seed_text += " " + output_word

print(seed_text)    
#my_texts = sequence_to_text(output_word)
#print(my_texts)

We are doing great economically as a great danger to our country that the u s is receiving is a gift to china tariffs trade trade deals and these thugs are not happy they got it wrong again fast and will never give us some of the things they are happy with


In [ ]:
print(prediction.shape)

(1, 10385)


In [ ]:
sequence_to_text([2])

['and']

In [ ]:
#sequence_to_text(y_data)
#y_data